In [1]:
pip install easyocr torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [2]:
!git clone https://github.com/JaidedAI/EasyOCR.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2750, done.
remote: Counting objects: 100% (846/846), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 2750 (delta 780), reused 756 (delta 756), pack-reused 1904 (from 1)
Receiving objects: 100% (2750/2750), 157.82 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (1689/1689), done.
Updating files: 100% (313/313), done.


In [3]:
%cd EasyOCR/trainer

/content/EasyOCR/trainer


In [ ]:
!unzip all_data/kk_train.zip -d all_data/kk_train

In [7]:
!rm -rf all_data/kk_train/__MACOSX/

In [21]:
#change from test import validation
#to
#from validation_test import validation

In [71]:
import importlib
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [59]:
cudnn.benchmark = True
cudnn.deterministic = False

In [60]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [61]:
opt = get_config("config_files/kazakh_dataset_config.yaml")

In [62]:
opt

{'number': '0123456789',
 'symbol': '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €',
 'lang_char': 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһі',
 'experiment_name': 'kazakh_dataset',
 'train_data': 'all_data',
 'valid_data': 'all_data/kk_val;',
 'manualSeed': 1111,
 'workers': 2,
 'batch_size': 16,
 'num_iter': 3000,
 'valInterval': 200,
 'saved_model': 'saved_models/kazakh_dataset',
 'FT': False,
 'optim': 'Adam',
 'lr': 1.0,
 'beta1': 0.9,
 'rho': 0.95,
 'eps': 1e-08,
 'grad_clip': 5,
 'select_data': 'kk_train',
 'batch_ratio': '1',
 'total_data_usage_ratio': 1.0,
 'batch_max_length': 50,
 'imgH': 200,
 'imgW': 200,
 'rgb': False,
 'contrast_adjust': 0.0,
 'sensitive': True,
 'PAD': True,
 'data_filtering_off': False,
 'Transformation': 'None',
 'FeatureExtraction': 'VGG',
 'SequenceModeling': 'BiLSTM',
 'Prediction': 'CTC',
 'num_fiducial': 20,
 'input_channel': 1,
 'output_channel': 256,
 'hidden_size': 256,
 'decode': 'greedy',
 'new_prediction': F

In [63]:
opt.data_filtering_off = True

In [11]:
!python train.py

cpu
I came to training
False
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['kk_train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: kk_train
all_data/kk_train/kk_train/.ipynb_checkpoints
all_data/kk_train/kk_train/
sub-directory:	/kk_train/kk_train/.ipynb_checkpoints	 num samples: 775
num total samples of kk_train: 775 x 1.0 (total_data_usage_ratio) = 775
num samples of kk_train per batch: 16 x 1.0 (batch_ratio) = 16
--------------------------------------------------------------------------------
Total_batch_size: 16 = 16
--------------------------------------------------------------------------------
dataset_root:    all_data/kk_val/kk_val	 dataset: /
all_data/kk_val/kk_val/.i

In [81]:
#check inference on one image

In [150]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import cv2
import numpy as np
from model import Model
from utils import  AttrDict


# Load trained model
model_path = "saved_models/kazakh_dataset/best_accuracy.pth"  # Update with your .pth file path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define model parameters (must match your training settings)
options = {
    "imgH": 200,
    "imgW": 200,
    "batch_max_length": 50,
    "hidden_size": 256,
    "Transformation": "None",
    "FeatureExtraction": "VGG",
    "SequenceModeling": "BiLSTM",
    "Prediction": "CTC",
    "num_fiducial": 20,
    "input_channel": 1,
    "output_channel": 256,
    "decode": "greedy",
    "sensitive": True,
    "num_class": len('0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһі')+1,
    "character": '0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһі'
}

# Initialize converter and model
converter = CTCLabelConverter(opt["character"])
num_class = len(converter.character) + 1
print(options)
options = AttrDict(options)
model = Model(options)
model = model.to(device)
# for name, param in model.named_parameters():
#       if 'localization_fc2' in name:
#           print(f'Skip {name} as it is already initialized')
#           continue
#       try:
#           if 'bias' in name:
#               init.constant_(param, 0.0)
#           elif 'weight' in name:
#               init.kaiming_normal_(param)
#       except Exception as e:  # for batchnorm.
#           if 'weight' in name:
#               param.data.fill_(1)
#           continue
model = torch.nn.DataParallel(model).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

print("Model Loaded Successfully!")


{'imgH': 200, 'imgW': 200, 'batch_max_length': 50, 'hidden_size': 256, 'Transformation': 'None', 'FeatureExtraction': 'VGG', 'SequenceModeling': 'BiLSTM', 'Prediction': 'CTC', 'num_fiducial': 20, 'input_channel': 1, 'output_channel': 256, 'decode': 'greedy', 'sensitive': True, 'num_class': 129, 'character': '0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһі'}
No Transformation module specified
Model Loaded Successfully!


<ipython-input-150-b00efbf8f9bd>:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [151]:
def preprocess_image(image_path, imgH=200, imgW=200):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read in grayscale
    print(img)
    img = cv2.resize(img, (imgW, imgH))
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = img / 255.0  # Normalize
    img = torch.FloatTensor(img).unsqueeze(0)  # Convert to tensor
    return img.to(device)

image_path = "all_data/kk_val/kk_val/sample_00800.jpg"  # Replace with your image path

image_tensor = preprocess_image(image_path)

[[220 227 237 ... 244 244 244]
 [228 234 241 ... 244 244 244]
 [239 242 245 ... 244 244 244]
 ...
 [244 244 244 ... 244 244 244]
 [244 244 244 ... 244 244 244]
 [244 244 244 ... 244 244 244]]


In [152]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("all_data/kk_val/kk_val/labels.csv", sep=',', usecols=["filename", "words"], keep_default_na=False)

print(df.head())


           filename                                          words
0  sample_00800.jpg          себебі жаман сен сіз және бар мен біз
1  sample_00801.jpg                                 үлкен және мен
2  sample_00802.jpg  әдемі үлкен себебі кіші жоқ ол олар керек бар
3  sample_00803.jpg                      Бүгін кешке үйде сөйледі.
4  sample_00804.jpg                         дүкенде қаламды жазды.


In [153]:
df[df['filename']=='sample_00800.jpg']['words'][0]

'себебі жаман сен сіз және бар мен біз'

In [155]:
with torch.no_grad():
    preds = model(image_tensor, '', False).log_softmax(2)
    print(preds)
    preds_size = torch.IntTensor([preds.size(1)])
    preds = preds.permute(1, 0, 2)
    _, preds_index = preds.max(2)

    pred_text = converter.decode_greedy(preds_index, preds_size)
    print(f"Predicted Text: {pred_text[0]}")
    print(f"Actual Text: {df[df['filename']=='sample_00800.jpg']['words'][0]}")

tensor([[[ -4.5766,  -7.8287, -10.1018,  ...,  -3.9274,  -9.0833,  -6.1471],
         [ -4.4149,  -9.6770, -10.1523,  ...,  -0.7803,  -9.3776,  -3.9097],
         [ -3.2630,  -8.7247,  -8.7799,  ...,  -4.0684,  -8.0905,  -4.1554],
         ...,
         [ -3.8536, -11.1915, -10.7718,  ...,  -8.2574, -11.9488,  -5.1285],
         [ -5.2833, -11.9566, -11.4150,  ...,  -8.2386, -11.4199,  -2.2865],
         [ -5.4003, -11.3062,  -9.3248,  ...,  -8.5152, -10.3034,  -4.9815]]],
       device='cuda:0')
Predicted Text: мүкіыды.
Actual Text: себебі жаман сен сіз және бар мен біз


In [115]:
#Now, lets train the model with pretrained ru weights

In [129]:
!python train.py

cuda
208
0123456789!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһі
0123456789!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһіABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzЂђЃѓЄєЇїЈјЉљЊњЋӢӣӮӯӢӣӮӯӯӮӯӢ
207
I came to training
False
Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['kk_train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: kk_train
all_data/kk_train/kk_train/.ipynb_checkpoints
all_data/kk_train/kk_train/
sub-directory:	/kk_train/kk_train/.ipynb_checkpoints	 num samples: 766
num total samples of kk_train: 766 x 1.0 (total_data_u

In [158]:
options = {
    "imgH": 200,
    "imgW": 200,
    "batch_max_length": 50,
    "hidden_size": 256,
    "Transformation": "None",
    "FeatureExtraction": "VGG",
    "SequenceModeling": "BiLSTM",
    "Prediction": "CTC",
    "num_fiducial": 20,
    "input_channel": 1,
    "output_channel": 256,
    "decode": "greedy",
    "sensitive": True,
    "character": """0123456789!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһіABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzЂђЃѓЄєЇїЈјЉљЊњЋӢӣӮӯӢӣӮӯӯӮӯӢ""",
    "num_class": 208
}

# Initialize converter and model
converter = CTCLabelConverter(opt["character"])
num_class = len(converter.character) + 1
print(options)
options = AttrDict(options)
model = Model(options)
model = model.to(device)
# for name, param in model.named_parameters():
#       if 'localization_fc2' in name:
#           print(f'Skip {name} as it is already initialized')
#           continue
#       try:
#           if 'bias' in name:
#               init.constant_(param, 0.0)
#           elif 'weight' in name:
#               init.kaiming_normal_(param)
#       except Exception as e:  # for batchnorm.
#           if 'weight' in name:
#               param.data.fill_(1)
#           continue
model = torch.nn.DataParallel(model).to(device)
model_path = "saved_models/kazakh_dataset_pretrained/best_accuracy.pth"
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

print("Model Loaded Successfully!")

{'imgH': 200, 'imgW': 200, 'batch_max_length': 50, 'hidden_size': 256, 'Transformation': 'None', 'FeatureExtraction': 'VGG', 'SequenceModeling': 'BiLSTM', 'Prediction': 'CTC', 'num_fiducial': 20, 'input_channel': 1, 'output_channel': 256, 'decode': 'greedy', 'sensitive': True, 'character': '0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ €АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯӘҒҚҢӨҰҮҺІабвгдеёжзийклмнопрстуфхцчшщъыьэюяәғқңөұүһіABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzЂђЃѓЄєЇїЈјЉљЊњЋӢӣӮӯӢӣӮӯӯӮӯӢ', 'num_class': 208}
No Transformation module specified
Model Loaded Successfully!


<ipython-input-158-ab734b0b891f>:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [159]:
#lets check same image

In [160]:
with torch.no_grad():
    preds = model(image_tensor, '', False).log_softmax(2)
    print(preds)
    preds_size = torch.IntTensor([preds.size(1)])
    preds = preds.permute(1, 0, 2)
    _, preds_index = preds.max(2)

    pred_text = converter.decode_greedy(preds_index, preds_size)
    print(f"Predicted Text: {pred_text}")
    print(f"Actual Text: {df[df['filename']=='sample_00800.jpg']['words'][0]}")

tensor([[[-9.6122e+00, -2.4388e+01, -2.8501e+01,  ..., -2.2421e+01,
          -2.2177e+01, -2.1270e+01],
         [-1.7489e+01, -2.1777e+01, -2.1406e+01,  ..., -1.8833e+01,
          -1.9314e+01, -1.8027e+01],
         [-3.9053e+00, -1.9620e+01, -1.8065e+01,  ..., -1.7381e+01,
          -1.7397e+01, -1.7685e+01],
         ...,
         [-5.4605e+00, -1.4291e+01, -1.4497e+01,  ..., -1.2717e+01,
          -1.5834e+01, -1.4765e+01],
         [-3.1404e+00, -1.1638e+01, -1.0026e+01,  ..., -1.1008e+01,
          -1.2132e+01, -1.2911e+01],
         [-4.3391e-05, -2.0782e+01, -1.8934e+01,  ..., -2.0910e+01,
          -2.1591e+01, -2.3941e+01]]], device='cuda:0')
Predicted Text: ['сенсеб бі кеқжен жолмақ әен ман']
Actual Text: себебі жаман сен сіз және бар мен біз


In [98]:
preds

tensor([[[ 2.5473, -0.7047, -2.9778,  ...,  3.1966, -1.9593,  0.9769],
         [ 2.8714, -2.3906, -2.8660,  ...,  6.5060, -2.0913,  3.3766],
         [ 3.4031, -2.0585, -2.1137,  ...,  2.5978, -1.4244,  2.5108],
         ...,
         [ 4.4192, -2.9187, -2.4990,  ...,  0.0154, -3.6760,  3.1443],
         [ 3.5949, -3.0784, -2.5368,  ...,  0.6396, -2.5417,  6.5917],
         [ 3.2047, -2.7012, -0.7197,  ...,  0.0899, -1.6984,  3.6236]]],
       device='cuda:0')

In [13]:
import json

# Load labels.json
with open("labels.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to EasyOCR format
with open("kazakh_dataset/annotations.txt", "w", encoding="utf-8") as f:
    for item in data:
        image_path = item["image_path"]
        text = item["text"]
        for bbox in item["bboxes"]:
            x1, y1, x2, y2 = bbox
            f.write(f"{image_path} {x1},{y1},{x2},{y2},{text}\n")


In [14]:
!mkdir -p easyocr/user_network/weights
!cd easyocr/user_network/weights
!wget https://www.jaided.ai/models/rus_g2.pth


--2025-02-23 10:21:02--  https://www.jaided.ai/models/rus_g2.pth
Resolving www.jaided.ai (www.jaided.ai)... 104.21.112.1, 104.21.96.1, 104.21.16.1, ...
Connecting to www.jaided.ai (www.jaided.ai)|104.21.112.1|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-02-23 10:21:03 ERROR 404: Not Found.



In [156]:
import easyocr

# Load Russian model to get existing characters
reader = easyocr.Reader(['ru'])


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

In [16]:
reader

In [157]:
image_path = 'all_data/kk_val/kk_val/sample_00800.jpg'  # Change this to your image path
results = reader.readtext(image_path)

# Print results
for bbox, text, confidence in results:
    print(f"Text: {text}, Confidence: {confidence:.2f}, BBox: {bbox}")

Text: себебч жаг, Confidence: 0.50, BBox: [[25, 27], [199, 27], [199, 65], [25, 65]]
Text: мен б1з, Confidence: 0.97, BBox: [[27, 61], [155, 61], [155, 99], [27, 99]]
